In [ ]:
import torch
import torch.backends
import torch.backends.cudnn
from torch.utils.data import Dataset, DataLoader,WeightedRandomSampler
import torchvision.models as models
import numpy as np
import torch.nn as nn
import warnings
import os
import cv2
import random
import math
import torch.nn.functional as F
from torch.optim import lr_scheduler
from utils.Custom_Dataset import CustomDataset
from utils.config import CFG
import matplotlib.pyplot as plt
from tqdm import tqdm
from models.RNN import CNNRNN
from models.LSTM import *
warnings.filterwarnings("ignore")

#12 시드 고정
seed = 12
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True  # 반복 가능성을 보장
torch.backends.cudnn.benchmark = False     # 성능 손해를 감수하고 일관성 유지

print(torch.device("cuda" if torch.cuda.is_available() else 'cpu'))
print(torch.__version__)


In [ ]:
np.random.seed(seed )

train_data=np.load('./data/image_train.npy')
train_labels=np.load('./data/label_train.npy')

indices = np.arange(train_data.shape[0]) ## 데이터 무작위 셔플링
np.random.shuffle(indices)


train_data = train_data[indices]
train_labels = train_labels[indices]



test_data=np.load('./data/image_valid.npy')
test_labels=np.load('./data/label_valid.npy')


학습데이터 8:2로 Train data와 Validation data로 구성

In [ ]:
ratio=0.8
num_data=len(train_data)

data_train=train_data[:int(ratio*num_data)]
labels_train=train_labels[:int(ratio*num_data)]

data_valid=train_data[int(ratio*num_data):]
label_valid=train_labels[int(ratio*num_data):]

print("Train_data: ", data_train.shape)
print("Train_label: ", labels_train.shape)

print("Valid_data: ", data_valid.shape)
print("Valid_label: ", label_valid.shape)


데이터 Downsampling

데드락 데이터를 다운샘플링하여 노말 데이터와 비슷한 데이터 수 유지

In [ ]:
image_list=[]
labels_list=[]
count=[0,0]
for idx,l in enumerate(labels_train):
    if l==1 and count[l]>237:
        continue
        
    if count[l]<238:

        count[l]+=1
        image_list.append(train_data[idx])
        labels_list.append(train_labels[idx])
data_train=np.array(image_list)

labels_train=np.array(labels_list)

print(count)

#데이터셋

전처리
+ RandomTranslate

    랜덤하게 이미지 상하좌우 이동 빈 여백 255로 채움
+ RandRotateView

    -30도에서 +30도로 랜덤하게 이미지 회전(사용)
+ RandBrightness

    랜덤하게 이미지의 밝기 변경
+ RandCutout

    랜덤하게 이미지에 구멍 6개 생성 구멍의 크기는 32x32
+ RandRotate90

    랜덤하게 이미지를 90도, 180도, 270도, 360도 회전
+ RandHorizontalFlip
    
    랜덤하게 이미지를 좌우반전(사용)
+ RandVerticalFlip

    랜덤하게 이미지를 수직 반전


In [ ]:

train_dataset=CustomDataset(data_train,labels_train,mode='Train')
valid_dataset=CustomDataset(data_valid,label_valid,mode='Valid')
test_dataset=CustomDataset(test_data,test_labels,mode='Valid')

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
n=20
t_image=train_dataset.__getitem__(n)[0]
t_label=train_dataset.__getitem__(n)[1]
print(t_image.shape)
print(t_label)
print(len(t_image))
figure,axis = plt.subplots(5,5,figsize=(12,12))

for i,ax in enumerate(axis.flat):
    
    Img_Pick = t_image[i]
    
    ax.set_xlabel(Img_Pick.shape)
    ax.imshow(Img_Pick.permute(1,2,0))

plt.tight_layout()
plt.show()

In [ ]:
def evaluate_model(model, valid_loader,dataset,criterion, device='cpu'):
    
    model.eval() 
    
    total_loss = 0.0
    correct_predictions = 0
    
    with torch.no_grad():
        for images, labels in tqdm(valid_loader):
            images = images.to(device)
            labels = labels.to(device).long()
           
            
           
            outputs,_ = model(images)
            

            loss = criterion(outputs, labels) 
            
            total_loss += loss
            
            predicted = torch.argmax(outputs, dim=1)
           
            
            
           
            correct_predictions += (predicted == labels).sum().item()
    print('outputs: ',predicted)
    print('label: ',labels)
    print("맞춘개수: ",correct_predictions,'/'+str(len(dataset)))
    avg_loss = total_loss / len(valid_loader) 
    accuracy = correct_predictions / len(dataset) 
    
    return avg_loss, accuracy

Learning Rate Shceduler

In [ ]:
class CosineAnnealingWarmUpRestarts(lr_scheduler._LRScheduler):
    def __init__(self, optimizer, T_0, T_mult=1, eta_max=0.1, T_up=0, gamma=1., last_epoch=-1):
        if T_0 <= 0 or not isinstance(T_0, int):
            raise ValueError("Expected positive integer T_0, but got {}".format(T_0))
        if T_mult < 1 or not isinstance(T_mult, int):
            raise ValueError("Expected integer T_mult >= 1, but got {}".format(T_mult))
        if T_up < 0 or not isinstance(T_up, int):
            raise ValueError("Expected positive integer T_up, but got {}".format(T_up))
        self.T_0 = T_0
        self.T_mult = T_mult
        self.base_eta_max = eta_max
        self.eta_max = eta_max
        self.T_up = T_up
        self.T_i = T_0
        self.gamma = gamma
        self.cycle = 0
        self.T_cur = last_epoch
        super(CosineAnnealingWarmUpRestarts, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.T_cur == -1:
            return self.base_lrs
        elif self.T_cur < self.T_up:
            return [(self.eta_max - base_lr) * self.T_cur / self.T_up + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.eta_max - base_lr) * (
                        1 + math.cos(math.pi * (self.T_cur - self.T_up) / (self.T_i - self.T_up))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.T_cur = self.T_cur + 1
            if self.T_cur >= self.T_i:
                self.cycle += 1
                self.T_cur = self.T_cur - self.T_i
                self.T_i = (self.T_i - self.T_up) * self.T_mult + self.T_up
        else:
            if epoch >= self.T_0:
                if self.T_mult == 1:
                    self.T_cur = epoch % self.T_0
                    self.cycle = epoch // self.T_0
                else:
                    n = int(math.log((epoch / self.T_0 * (self.T_mult - 1) + 1), self.T_mult))
                    self.cycle = n
                    self.T_cur = epoch - self.T_0 * (self.T_mult ** n - 1) / (self.T_mult - 1)
                    self.T_i = self.T_0 * self.T_mult ** (n)
            else:
                self.T_i = self.T_0
                self.T_cur = epoch

        self.eta_max = self.base_eta_max * (self.gamma ** self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [ ]:

print("SEED: ",seed)

base_net = CNNLSTM().to(CFG['device'])
optimizer=torch.optim.Adam(base_net.parameters(),lr=0,betas=(0.5, 0.999), amsgrad=True,weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()
best_accuracy = 0.0
sheduler=CosineAnnealingWarmUpRestarts(optimizer,T_0=10,T_mult=1,eta_max=0.001,T_up=3,gamma=0.5)

loss_count=0
test_accuracy_list=[]
for epoch in range(1,CFG['EPOCH']+1):

    base_net.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for i,(image,label) in enumerate(tqdm(train_dataloader)):
        images=image.to(CFG['device'])
        labels=label.to(CFG['device']).long()
        
        
    
        
        
        optimizer.zero_grad()

        outputs,_ = base_net(images)


        
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        running_loss += loss  # 배치 손실 누적
        
        predicted = torch.argmax(outputs, dim=1)
        correct_predictions += (predicted == labels).sum().item()


        
    avg_train_loss = running_loss / len(train_dataloader)
    train_accuracy = correct_predictions/len(train_dataset)
    
    avg_valid_loss, valid_accuracy = evaluate_model(base_net, valid_dataloader, valid_dataset,criterion, CFG['device'])
    sheduler.step()
    curve_lr=optimizer.param_groups[0]['lr']
    print(f'Epoch: {epoch} LEARNING RATE: ',{curve_lr})
    print(f'Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')
    print(f'Validation Loss: {avg_valid_loss:.4f}, Validation Accuracy: {valid_accuracy:.4f}')
    print('COUNT: ',loss_count)
    
    
    torch.save(base_net, './models/'+str(epoch)+'_LSTM_'+str(valid_accuracy)[:5]+'_best_model.pt')

